# Create timetable calendar from Excel spreadsheet

This code takes a named cell range in an Excel spreadsheet and produces a ics calendar file.

In [1]:
import numpy as np
import pandas as pd
import openpyxl
from icalendar import Calendar, Event, vText, vCalAddress

Documentation for iCalendar library can be found at: http://icalendar.readthedocs.io/en/latest/

In [2]:
# Import function to read data from Excel spreadsheet and load into Pandas dataframe
import phjReadData

import importlib
importlib.reload(phjReadData)

<module 'phjReadData' from '/Users/philjones/Dropbox/phjPythonModules/PHJ-GitHub/phj_python_modules/phjReadData.py'>

## Enter paths and filenames

In [3]:
phjPath = '/Users/philjones/Documents/python_projects/python34_example_environment_2015-10-30'
phjExcelFilename = 'Timetable spreadsheet example.xlsx'
phjExcelRangeName = 'timetable'

# The following will be used to label calendar events
phjCourseHeader = 'COURSE NAME 2016-17'

# The following file will be created
phjICSFilename = phjCourseHeader + ' calendar.ics'

## Read data from Excel spreadsheet

In [4]:
phjTimetableDF = phjReadData.phjReadDataFromExcelNamedCellRange(phjExcelFileName = '/'.join([phjPath,phjExcelFilename]),
                                                                phjExcelRangeName = phjExcelRangeName,
                                                                phjDatetimeFormat = "%Y-%m-%d",
                                                                phjMissingValue = np.nan,
                                                                phjHeaderRow = True,
                                                                phjPrintResults = True)


List of iterable properties:
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'destinations', 'localSheetId', 'name', 'repr_format', 'scope', 'str_format', 'value']

List of tuples of named ranges and cell ranges
[(<ReadOnlyWorksheet "Sheet1">, '$A$5:$L$31')]

First tuple of named ranges and cell range
(<ReadOnlyWorksheet "Sheet1">, '$A$5:$L$31')

Cell range
$A$5:$L$31

Local Sheet ID
<ReadOnlyWorksheet "Sheet1">

First row (containing variable names) has been removed from the data.
var 1 :  date
var 2 :  day_number
var 3 :  week_number
var 4 :  time_string
var 5 :  start_time
var 6 :  end_time
var 7 :  number_hours
var 8 :  room
var 9 :  type
var 10 :  title
var 11 :  staff
var 12 :  comments

Imported data
--------

In [5]:
phjTimetableDF['date'] = pd.to_datetime(phjTimetableDF['date'])

Remove rows where no time_string information is set (i.e. no lecture takeing place)

In [6]:
phjTimetableDF = phjTimetableDF[phjTimetableDF['time_string'].notnull()].reset_index(drop = True)

with pd.option_context('display.max_rows', 10, 'display.max_columns', 5):
    print(phjTimetableDF)

         date  day_number   ...         staff comments
0  2016-10-11           2   ...     A.N.Other      NaN
1  2016-10-12           3   ...     A.N.Other      NaN
2  2016-10-14           5   ...     A.N.Other      NaN
3  2016-10-18           2   ...     A.N.Other      NaN
4  2016-10-19           3   ...     A.N.Other      NaN
..        ...         ...   ...           ...      ...
16 2016-10-25           2   ...           NaN      NaN
17 2016-10-25           2   ...           NaN      NaN
18 2016-10-28           5   ...           NaN      NaN
19 2016-10-28           5   ...           NaN      NaN
20 2016-10-28           5   ...           NaN      NaN

[21 rows x 12 columns]


In [7]:
cal = Calendar()

In [8]:
cal.add('prodid', '-//Python iCalendar timetable//')
cal.add('version', '2.0')
organizer = vCalAddress('MAILTO:p.h.jones@liverpool.ac.uk')
organizer.params['cn'] = vText('Phil Jones')
organizer.params['role'] = vText('Lecturer')

# UID will be (arbitrarily) constucted using datetime of start event, title
# of event and the label given below to provide human-readable code.
# Actually, by creating a UID that will be duplicated if the script is re-run,
# the old items in the calendar will be replaced if the .ics file is added.
# Therefore, didn't create a UID using datetime of 'now'.
# Equally, it is important that different event that overlap don't have the
# same UiD, hence include the title of the event to ensure UIDs are different.
phjEventUIDLabel = 'vsci100_2016-17_ephw_sem1@liv.ac.uk'

for index, row in phjTimetableDF.iterrows():
    
    # Create event in calendar object
    event = Event()
    
    # Add properties to event
    event.add('organizer', organizer)
    
    # See above note about creating UIDs for events
    event.add('uid', '/'.join([pd.to_datetime(row['date'].strftime('%Y-%m-%d') + ' ' + row['start_time']).strftime('%Y%m%d%H%M%S'),
                               row['title'],
                               phjEventUIDLabel]))
    
    if pd.isnull(row['staff']):
        event.add('summary', phjCourseHeader + ' ' + row['type'] + ' - ' + row['title'])
    else:
        event.add('summary', phjCourseHeader + ' ' + row['type'] + ' - ' + row['title'] + ' (' + row['staff'] + ')')
        
    event.add('dtstart', pd.Timestamp(row['date'].strftime('%Y-%m-%d') + ' ' + row['start_time'], tz='Europe/London'))
    event.add('dtend',   pd.Timestamp(row['date'].strftime('%Y-%m-%d') + ' ' + row['end_time'], tz='Europe/London'))
    event.add('location', row['room'])
    
    # Add event to calendar
    cal.add_component(event)

In [9]:
print(cal.to_ical())

b'BEGIN:VCALENDAR\r\nVERSION:2.0\r\nPRODID:-//Python iCalendar timetable//\r\nBEGIN:VEVENT\r\nSUMMARY:COURSE NAME 2016-17 LECTURE - Lecture 1 (A.N.Other)\r\nDTSTART;TZID=Europe/London;VALUE=DATE-TIME:20161011T160000\r\nDTEND;TZID=Europe/London;VALUE=DATE-TIME:20161011T170000\r\nUID:20161011160000/Lecture 1/vsci100_2016-17_ephw_sem1@liv.ac.uk\r\nLOCATION:TBA\r\nORGANIZER;CN="Phil Jones";ROLE=Lecturer:MAILTO:p.h.jones@liverpool.ac.uk\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nSUMMARY:COURSE NAME 2016-17 LECTURE - Lecture 2 (A.N.Other)\r\nDTSTART;TZID=Europe/London;VALUE=DATE-TIME:20161012T110000\r\nDTEND;TZID=Europe/London;VALUE=DATE-TIME:20161012T120000\r\nUID:20161012110000/Lecture 2/vsci100_2016-17_ephw_sem1@liv.ac.uk\r\nLOCATION:TBA\r\nORGANIZER;CN="Phil Jones";ROLE=Lecturer:MAILTO:p.h.jones@liverpool.ac.uk\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nSUMMARY:COURSE NAME 2016-17 LECTURE - Lecture 3 (A.N.Other)\r\nDTSTART;TZID=Europe/London;VALUE=DATE-TIME:20161014T100000\r\nDTEND;TZID=Europe/London;VALUE=

Write ical of all events in timetable to file in same location as original Excel file

In [ ]:
f = open('/'.join([phjPath,phjICSFilename]),'wb')
f.write(cal.to_ical())
f.close()